# Restore Database from S3 Snapshot

Request a link to an S3 Snapshot from an Awesome Foundation Website admin and paste it into the form below.

In [ ]:
db_password = os.getenv('POSTGRES_PASSWORD')

%env PGPASSWORD=$db_password

import hashlib
import io
import os
import subprocess
from subprocess import Popen, PIPE
import sys
from tempfile import TemporaryFile, NamedTemporaryFile
import urllib.request

from IPython.display import display
from ipywidgets import widgets
import requests
import shlex
from tqdm import tqdm

class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)


def download_url(url, output_path):
    with DownloadProgressBar(unit='B',
                             unit_scale=True,
                             miniters=1,
                             desc=url.split('/')[-1].split('?')[0]) as t:
        urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)

def restore_database(snapshot_url):
    os.makedirs('snapshots', exist_ok=True)
    snapshot_url_hash = hashlib.sha256(snapshot_url.encode('utf-8')).hexdigest()
    snapshot_file_path = f"snapshots/{snapshot_url_hash}.pgdump"

    if not os.path.exists(snapshot_file_path):
      download_url(snapshot_url, snapshot_file_path)
    
    username = os.getenv('POSTGRES_USER')
    password = os.getenv('POSTGRES_PASSWORD')
    database_name = os.getenv('DB_NAME')
    host = 'db'

    !psql -h {host} -d {database_name} -U {username} -c "DROP DATABASE \"$DATABASE\";"
    !pg_restore -h {host} -d {database_name} -U {username} -w {snapshot_file_path}

snapshot_url_input = widgets.Text(placeholder="Please Enter Snapshot URL and Hit Enter")
button = widgets.Button(description="Restore database!")

display(snapshot_url_input, button)

def on_submit(wdgt):
    restore_database(snapshot_url_input.value)

button.on_click(on_submit)